<a href="https://colab.research.google.com/github/pmckim1/Capstone/blob/main/Event_Extraction_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This File is the code used to run the Giveme5W1H algorithm 
** This code was run on AWS using tmux and required stanford core NLP server to be running in order to run correctly 

In [ ]:
import csv
import json
# from google.colab import drive

import pandas as pd
import numpy as np
from Giveme5W1H.extractor.document import Document
from Giveme5W1H.extractor.extractor import MasterExtractor
from Giveme5W1H.extractor.preprocessors.preprocessor_core_nlp import Preprocessor

# drive.mount('/content/drive')

# Get the absolute path of the current folder
# abspath_curr = '/content/drive/My Drive/capstone/capstone/Data/Guardian/'
abspath_curr="~/"

df_full_input = pd.read_csv(abspath_curr +'litte_set_2_to_4.csv', header=0)
df_partially_processed = pd.read_csv(abspath_curr +'stream-topped_2_to_4_saved.csv', header=0)

fieldnames = df_full_input.columns.values.tolist()

fieldnames.append('top_who_answer')
fieldnames.append('top_what_answer')
fieldnames.append('top_when_answer')
fieldnames.append('top_where_answer')
fieldnames.append('top_why_answer')
fieldnames.append('top_how_answer')

# df = pd.merge(df_full_input, df_partially_processed, on=["t", "Id"])
df = pd.merge(
    df_full_input,
    df_partially_processed[
      ['t', 'Id', 'top_who_answer','top_what_answer', 'top_when_answer', 'top_where_answer', 'top_why_answer', 'top_how_answer']
    ],
    on=['t', 'Id'],
    how='left'
)
print(df_full_input.head(5))
print(df_full_input.tail(5))
print(len(df_full_input))
print(df_partially_processed.head(5))
print(df_partially_processed.tail(5))
print(len(df_partially_processed))
print(df.head(5))
print(df.tail(5))
print(len(df))

df["top_who_answer"]=df.top_who_answer.astype(str)

stream_csvfile = open('./stream-topped_2_to_4.csv', 'w', newline='')
stream_csv_writer = csv.DictWriter(stream_csvfile, fieldnames=fieldnames)
stream_csv_writer.writeheader()

ctr = 0
size = len(df)
def extract_5w(row):
  global ctr, size
  ctr += 1
  print("Working on row {} of {}: t {} Id {}.".format(ctr, size, row['t'], row['Id']))
  try:
    if row['top_who_answer'] is None or \
        row['top_who_answer'] == "NaN" or \
        len(row['top_who_answer']) < 5:
      preprocessor = Preprocessor('http://localhost:9000')
      extractor = MasterExtractor(preprocessor=preprocessor)
      text = row['text']
      date_publish = row['pub_date_dto']
      doc = Document.from_text(text, date_publish)
      doc = extractor.parse(doc)
      row['top_who_answer'] = doc.get_top_answer('who').get_parts_as_text()
      row['top_what_answer'] = doc.get_top_answer('what').get_parts_as_text()
      row['top_when_answer'] = doc.get_top_answer('when').get_parts_as_text()
      row['top_where_answer'] = doc.get_top_answer('where').get_parts_as_text()
      row['top_why_answer'] = doc.get_top_answer('why').get_parts_as_text()
      row['top_how_answer'] = doc.get_top_answer('how').get_parts_as_text()
    else:
      print("Row already processed.")
    # print(row['c1'], row['c2'])
  except Exception as e:
    print("Unable to process document")
    print(e)
    print("Saving what has been found, and continuing to next document")
  finally:
    print("Saving row to stream-csv.")
    try:
      stream_csv_writer.writerow(json.loads(row.to_json()))
    except Exception as e:
      print("Could not save row to stream-csv.")
    return row

try:
  df = df.apply(lambda row: extract_5w(row), axis=1)
except Exception as e:
  print("Failure during extract5w")
  print(e)
  print("Saving curring to disk, abandoning the rest.")
print("Head:")
print(df.head(5))
print(df.tail(5))

stream_csvfile.close()
print("Saving ./topped_2_to_4.csv")
df.to_csv("./topped_2_to_4.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
22155


'\nfieldnames = df.columns.values.tolist()\n\nfieldnames.append(\'top_who_answer\')\nfieldnames.append(\'top_what_answer\')\nfieldnames.append(\'top_when_answer\')\nfieldnames.append(\'top_where_answer\')\nfieldnames.append(\'top_why_answer\')\nfieldnames.append(\'top_how_answer\')\n\nstream_csvfile = open(\'./stream-topped_5_to_9.csv\', \'w\', newline=\'\')\nstream_csv_writer = csv.DictWriter(stream_csvfile, fieldnames=fieldnames)\nstream_csv_writer.writeheader()\n\nctr = 0\nsize = len(df)\ndef extract_5w(row):\n  global ctr, size\n  ctr += 1\n  print("Working on row {} of {}: t {} Id {}.".format(ctr, size, row[\'t\'], row[\'Id\']))\n  try:\n    preprocessor = Preprocessor(\'http://localhost:9000\')\n    extractor = MasterExtractor(preprocessor=preprocessor)\n    text = row[\'text\']\n    date_publish = row[\'pub_date_dto\']\n    doc = Document.from_text(text, date_publish)\n    doc = extractor.parse(doc)\n    row[\'top_who_answer\'] = doc.get_top_answer(\'who\').get_parts_as_text()\n